## 1. Set the project and import Charcrete database

In [204]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as ba
import bw_processing as bwp

# General libraries
import numpy as np
import pandas as pd
import math
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
import os

# Visualization
import plotly.graph_objects as go

# Uncertainties
import stats_arrays as sa

In [205]:
bd.projects.set_current("ei38-teaching-25")

## 2. Importing Charcrete database

In [206]:
#finding the relative path
filepath = os.getcwd()+"/data/Charcrete_database_38.xlsx"

In [207]:
if "Charcrete_38" in bd.databases:
    del bd.databases["Charcrete_38"]
    
# 1. Specify filepath to your foreground inventories.
charcrete_path = filepath
# 2. Create an instance of a class that contains basic methods for importing a database from an excel file.
charcrete = bi.ExcelImporter(charcrete_path)  
# 3. `apply_strategies` is one of such basic methods, it makes sure units, locations, etc are in correct format.
charcrete.apply_strategies()
# 4. Next step is to link your foreground exchanges to existing databases by matching relevant exchanges fields.
charcrete.match_database("biosphere3", fields=("name", "unit", "categories"))
charcrete.match_database("ei 3.8 cutoff", fields=("name", "location", "unit"))
#charcrete.metadata.pop(None)  # Remove metadata None entry. TODO
# 5. If everything is linked, write database so that it is saved in your project.
if charcrete.all_linked:
    charcrete.write_database()

Extracted 2 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.96 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
7 datasets
43 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [208]:
bd.databases

Databases dictionary with 7 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리

## 3. Check LCA on charcrete to ensure everything is working fine

In [275]:
char_db = bd.Database("Charcrete_38")
ei_db = bd.Database("ei 3.8 cutoff")

In [210]:
char_p = [act for act in char_db if "Charcrete" in act['name']][0]

In [211]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
method = bd.Method(ipcc)

In [212]:
def calculateLCA(act, amount, method):
    fu, data_objs, _ = bd.prepare_lca_inputs({act: amount}, method=ipcc)
    lca = bc.LCA(fu, data_objs=data_objs)
    lca.lci()
    lca.lcia()
    return lca.score

In [213]:
calculateLCA(char_p, 1, ipcc)

177.6456230224958

## 4. Scenarios with datapackage

char_db = charcrete database

ei_db = ecoinvent database

In [215]:
char_init = [
    act for act in char_db 
    if 'Charcrete' in act['name'] 
    and '5' in act['name'] 
    and '7' not in act['name']
][0]

In [216]:
char_df = pd.read_excel(filepath, sheet_name = 'scenario')

In [217]:
[exc['name'] for exc in char_init.technosphere()]

['Biochar production',
 'market for gravel, crushed',
 'market for electricity, medium voltage',
 'market for tap water',
 'market for sand',
 'market for plasticiser, for concrete, based on sulfonated melamine formaldehyde',
 'market for diesel, burned in building machine',
 'market for cement, Portland']

In [178]:
len(char_init.technosphere())

8

In [223]:
id_act_dict = {}
for exc in char_init.technosphere(): #adding the exchange technosphere
    ind = exc.input.id
    na = exc['name']
    id_act_dict[na]=ind
for exc in char_init.biosphere(): #adding the exchange biosphere
    ind = exc.input.id
    na = exc['name']
    id_act_dict[na]=ind
id_act_dict[char_init['name']]=char_init.id #adding the production
id_act_dict

{'Biochar production': 33100,
 'market for gravel, crushed': 15962,
 'market for electricity, medium voltage': 10117,
 'market for tap water': 15716,
 'market for sand': 5008,
 'market for plasticiser, for concrete, based on sulfonated melamine formaldehyde': 6509,
 'market for diesel, burned in building machine': 10200,
 'market for cement, Portland': 13050,
 'Carbon dioxide, to soil or biomass stock': 3891,
 'Charcrete production 5%': 33101}

In [224]:
id_act_dict['Biochar production']

33100

In [225]:
lg = len (char_df.columns) #number of culum in the dataframe

In [226]:
char_df['Activity name'][0]

'Charcrete production 5%'

In [227]:
list_id=[]
for e in char_df['Activity name']:
    list_id.append(id_act_dict[e])           
list_id

[33101, 33100, 15962, 10117, 15716, 5008, 6509, 10200, 13050, 3891]

In [228]:
#adding a colum for id
char_df.insert(loc = lg-1, value = list_id,column = 'id')

In [229]:
char_df

,Activity name,reference product,location,Charcrete production 5%,Charcrete production 7.5%,Charcrete production 10%,unit,database,id,type
0,Charcrete production 5%,Charcrete production 5%,SE,1.000,1.000,1.000,cubic meter,Charcrete_38,33101,production
1,Biochar production,Biochar production,SE,0.010,0.015,0.020,ton,Charcrete_38,33100,technosphere
2,"market for gravel, crushed","gravel, crushed",CH,1000.000,1000.000,1000.000,kilogram,ei 3.8 cutoff,15962,technosphere
3,"market for electricity, medium voltage","electricity, medium voltage",SE,5.500,5.500,5.500,kilowatt hour,ei 3.8 cutoff,10117,technosphere
4,market for tap water,tap water,Europe without Switzerland,156.000,156.000,156.000,kilogram,ei 3.8 cutoff,15716,technosphere
5,market for sand,sand,CH,938.000,938.000,938.000,kilogram,ei 3.8 cutoff,5008,technosphere
6,"market for plasticiser, for concrete, based on...","plasticiser, for concrete, based on sulfonated...",GLO,3.200,5.000,6.600,kilogram,ei 3.8 cutoff,6509,technosphere
7,"market for diesel, burned in building machine","diesel, burned in building machine",GLO,0.200,0.200,0.200,MJ,ei 3.8 cutoff,10200,technosphere
8,"market for cement, Portland","cement, Portland",Europe without Switzerland,186.800,180.000,173.400,kilogram,ei 3.8 cutoff,13050,technosphere
9,"Carbon dioxide, to soil or biomass stock",NaN,NaN,3.256,4.884,6.512,kilogram,biosphere3,3891,biosphere


In [231]:
char_df['id'][0]

33101

In [248]:
#generation of the tuple list for indices
list_ind_t = []
list_data_t = []
list_flip_t = []
list_ind_b = []
list_data_b = []

for i in range (len(char_df)):
    ind_t = (char_df['id'][i],char_df['id'][0])
    data = (char_df['Charcrete production 5%'][i],char_df['Charcrete production 7.5%'][i],char_df['Charcrete production 10%'][i])
    if char_df['type'][i]=='technosphere' or char_df['type'][i]=='production': #first let's do the technosphere flows
        if char_df['id'][i]==char_df['id'][0]:
            flip = False
        else:
            flip = True
        list_ind_t.append(ind_t)
        list_data_t.append(data)
        list_flip_t.append(flip)
    else:
        assert char_df['type'][i]=='biosphere'
        list_ind_b.append(ind_t)
        list_data_b.append(data)


In [252]:
list_ind_t

[(33101, 33101),
 (33100, 33101),
 (15962, 33101),
 (10117, 33101),
 (15716, 33101),
 (5008, 33101),
 (6509, 33101),
 (10200, 33101),
 (13050, 33101)]

In [253]:
list_ind_b

[(3891, 33101)]

In [260]:
list_data_b

[(3.2560000000000002, 4.884, 6.5120000000000005)]

In [256]:
np.array(list_flip_t)

array([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [261]:
dp_scenarios = bwp.create_datapackage(sequential = True)
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array(list_ind_t, dtype=bwp.INDICES_DTYPE),
    data_array=np.array(list_data_t),
    flip_array=np.array(list_flip_t),
#seed_overwrite=42 should not be used or it would mix the column order of the array
)
dp_scenarios.add_persistent_array(
    matrix='biosphere_matrix',
    indices_array=np.array(list_ind_b, dtype=bwp.INDICES_DTYPE),
    data_array=np.array(list_data_b),
)

In [267]:
fu, data_objs, _ = bd.prepare_lca_inputs({char_init: 1}, method=ipcc)

In [269]:
data_objs

In [276]:
char_init

'Charcrete production 5%' (cubic meter, SE, None)

In [271]:
fu, data_objs, _ = bd.prepare_lca_inputs({char_init: 1}, method=ipcc)

lca_b = bc.LCA(
    fu,
    data_objs=[*data_objs,dp_scenarios], # the "*" unpack whatever is inside
    use_distributions=False,
    use_arrays=True,
#     seed_override=42,  # Seed should not be used
)
lca_b.lci()
lca_b.lcia()
lca_b.score

177.6456203790934

In [273]:
lca_b.keep_first_iteration() #if you don't put that, the first iteration will be overwritten
iterations = 3
scores_b = [lca_b.score for _ in zip(range(iterations), lca_b)]
scores_b

[177.6456203790934, 173.55581965877943, 169.37472971580868]

# validation step

char_db = charcrete database

ei_db = ecoinvent database

In [278]:
bd.databases

Databases dictionary with 7 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리

In [283]:
scores_b

[177.6456203790934, 173.55581965877943, 169.37472971580868]

In [281]:
char_act_list = [act for act in  char_db if 'Charcrete production' in act['name']]

In [284]:
lca_score_list_validation=[]
for act in char_act_list :
    fu, data_objs, _ = bd.prepare_lca_inputs({act: 1}, method=ipcc)
    lca_b = bc.LCA(
        fu,
        data_objs=[*data_objs,dp_scenarios], # the "*" unpack whatever is inside
        use_distributions=False,
        use_arrays=False,
    #     seed_override=42,  # Seed should not be used
    )
    lca_b.lci()
    lca_b.lcia()
    lca_score_list_validation.append((lca_b.score,act['name']))
lca_score_list_validation

[(173.5558197613203, 'Charcrete production 7.5%'),
 (169.37472406675374, 'Charcrete production 10%'),
 (177.6456230224958, 'Charcrete production 5%')]

## 5.WiP on correlated samples

In [18]:
[act for act in ei_db if 'concrete' in act['name']][0].as_dict()

{'comment': 'Hardening is assumed to be by air-drying. It is assumed that there is wastewater and solid waste, but as no exact description on the composition of the wastewater and solid waste is available a proxy for each ("treatment, concrete production effluent, to wastewater treatment, class 3" and "disposal, municipal solid waste, 22.9% water, to municipal incineration") is used. The lifespan of the plant is assumed to be 50 years.\nLightweight concrete block. Image source: http://keramzit.by/data/pictures/poduct/block1.png\nImage: https://db3.ecoinvent.org/images/0b755024-3429-4acb-a241-cffdc608c3ff\nGeography:  For some exchanges CH, RER and GLO modules have been used as proxy.\nTechnology:  The raw material mixture (aggregates, cement and water) of lightweight concrete blocks are poured into a mould then squeezed and finaly hardened by air (assumption for this study)',
 'classifications': [('EcoSpold01Categories',
   'construction materials/concrete'),
  ('ISIC rev.4 ecoinvent',

In [19]:
concrete_ei_20MPa = [act for act in ei_db if "concrete production 20MPa" in act["name"]]
concrete_ei_20MPa

['concrete production 20MPa' (cubic meter, CA-QC, None),
 'concrete production 20MPa' (cubic meter, ZA, None),
 'concrete production 20MPa' (cubic meter, RoW, None),
 'concrete production 20MPa' (cubic meter, North America without Quebec, None)]

In [37]:
exc_list = []
for act in concrete_ei_20MPa :
     for exc in act.technosphere():
         if exc['unit']== 'kilogram': 
            exc_list.append(exc['name'])
         else: 
            pass
exc_list = set(exc_list)
exc_list

{'acetic acid, without water, in 98% solution state',
 'alkylbenzene sulfonate, linear, petrochemical',
 'cement, Portland',
 'cement, portland fly ash cement 21-35%',
 'chemical, organic',
 'ethylene oxide',
 'gravel, round',
 'lubricating oil',
 'sand',
 'steel, low-alloyed, hot rolled',
 'synthetic rubber',
 'tap water',
 'waste concrete'}

In [54]:
len(concrete_ei_20MPa)

4

In [47]:
density_mapping = []
for act in concrete_ei_20MPa :
    d=0
    for exc in act.technosphere():
        if exc['unit']== 'kilogram': 
            d = d + exc['amount']     
        else: 
            pass
    density_mapping.append([act['name'],d])
density_mapping

[['concrete production 20MPa', 2330.772867449367],
 ['concrete production 20MPa', 2415.9284602953603],
 ['concrete production 20MPa', 2415.9284602953594],
 ['concrete production 20MPa', 2338.1568988804133]]

In [73]:
def ComputeDensity(act_list_in_cubic_meter):
    density_mapping = []
    for act in act_list_in_cubic_meter :
        d=0
        try : 
            act['unit']=='cubic meter'
        except ValueError:
            raise ValutError("activity unit has to be cubic meter")
        for exc in act.technosphere():
            if exc['unit']== 'kilogram': 
                d = d + exc['amount']
            else:
                pass
        density_mapping.append([act['name'], act['location'],d])
    return pd.DataFrame(density_mapping , columns = ["Activity Name", "Location","density"])

In [74]:
ComputeDensity(concrete_ei_20MPa)

,Activity Name,Location,density
0,concrete production 20MPa,CA-QC,2330.772867
1,concrete production 20MPa,ZA,2415.928460
2,concrete production 20MPa,RoW,2415.928460
3,concrete production 20MPa,North America without Quebec,2338.156899


In [105]:
l = [
act for act in ei_db 
    if 'concrete' in act['name'] 
    and act['unit']=='cubic meter' 
    and 'market' not in act['name']
    and 'treatment' not in act['name']
    and 'slab' not in act['name']
]

In [106]:
density_table = ComputeDensity(l)

In [107]:
density_table.sort_values(by = 'density')

,Activity Name,Location,density
64,"concrete production, 40MPa, ready-mix, with ce...",RoW,2182.69546
21,"concrete production, 40MPa, ready-mix, with ce...",BR,2182.69546
59,"lean concrete production, with cement CEM II/B",CH,2244.80000
9,"lean concrete production, with cement CEM II/A",CH,2244.80000
68,"lean concrete production, with cement CEM II/A",RoW,2244.80000
...,...,...,...
55,"concrete production, 40MPa, ready-mix, with ce...",PE,2514.71400
49,"concrete production, 40MPa, ready-mix, with ce...",RoW,2514.71400
67,"concrete production, 40MPa, ready-mix, with ce...",RoW,2514.71400
60,"concrete production, 20MPa, ready-mix, with ce...",PE,2552.71400


In [91]:
for i in range (0:len(density_table))
    if density_table.loc(i)['density']

102

In [110]:
bd.databases

Databases dictionary with 7 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리